In [1]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [3]:
df = pd.read_csv("WR-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 1009

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,drafted
0,2008,brandon breazell,UCLA,WR,71.63,160,4.47,NaN,30.5,115.0,NaN,6.97,N
1,1997,brian roberson,Fresno State (CA),WR,68.80,161,4.65,NaN,30.5,115.0,4.14,7.36,N
2,1998,tutu atwell,Minnesota,WR,68.10,164,4.51,NaN,32.5,117.0,4.22,7.34,N
3,1999,siaha burley,Central Florida,WR,71.00,166,4.53,NaN,34.0,113.0,4.16,NaN,N
4,1997,pearce pegross,Baylor,WR,69.40,170,4.63,NaN,31.5,116.0,4.29,7.34,N


In [4]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
for index, row in df.iterrows():
    if (df.loc[index,"hasstats"] == ""):
        x = row["name"]
        browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
        if (len(browser.find_by_css("table#receiving")) > 0):
            elem = len(browser.find_by_css("td[data-stat='rec']"))
            df.loc[index, "Crec"] = browser.find_by_css("td[data-stat='rec']")[elem-1].text
            df.loc[index, "Cyds"] = browser.find_by_css("td[data-stat='rec_yds']")[elem-1].text
            df.loc[index, "CTD"] = browser.find_by_css("td[data-stat='rec_td']")[elem-1].text
            
            if (len(browser.find_by_css("ul#bling")) > 0):
                df.loc[index,"hasbling"] = 1
            else:
                df.loc[index,"hasbling"] = 0
            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                df.loc[index,"allamerican"] = 1
            else:
                df.loc[index,"allamerican"] = 0
            if (len(browser.find_by_css("li.bling_hof")) > 0):
                df.loc[index,"heisman"] = 1
            else:
                df.loc[index,"heisman"] = 0
            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                df.loc[index,"hasawards"] = 1
            else:
                df.loc[index,"hasawards"] = 0
        elif (len(browser.find_by_css("div.search-item")) == 0):
            df.loc[index,"hasbling"] = 0
            df.loc[index,"allamerican"] = 0
            df.loc[index,"heisman"] = 0
            df.loc[index,"hasawards"] = 0
        else:
            listed = browser.find_by_css("div.search-item")
            
            print()
            
            for x in listed:
                if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                    x.find_by_css("div.search-item-name a").click()
                    if (len(browser.find_by_css("table#receiving")) > 0):
                        elem = len(browser.find_by_css("td[data-stat='rec']"))
                        df.loc[index, "Crec"] = browser.find_by_css("td[data-stat='rec']")[elem-1].text
                        df.loc[index, "Cyds"] = browser.find_by_css("td[data-stat='rec_yds']")[elem-1].text
                        df.loc[index, "CTD"] = browser.find_by_css("td[data-stat='rec_td']")[elem-1].text
                        
                        if (len(browser.find_by_css("ul#bling")) > 0):
                            df.loc[index,"hasbling"] = 1
                        else:
                            df.loc[index,"hasbling"] = 0
                        if (len(browser.find_by_css("li.bling_all_star")) > 0):
                            df.loc[index,"allamerican"] = 1
                        else:
                            df.loc[index,"allamerican"] = 0
                        if (len(browser.find_by_css("li.bling_hof")) > 0):
                            df.loc[index,"heisman"] = 1
                        else:
                            df.loc[index,"heisman"] = 0
                        if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                            df.loc[index,"hasawards"] = 1
                        else:
                            df.loc[index,"hasawards"] = 0
                        break
                    else:
                        df.loc[index,"hasbling"] = 0
                        df.loc[index,"allamerican"] = 0
                        df.loc[index,"heisman"] = 0
                        df.loc[index,"hasawards"] = 0
                        break
                else:
                    df.loc[index,"hasbling"] = 0
                    df.loc[index,"allamerican"] = 0
                    df.loc[index,"heisman"] = 0
                    df.loc[index,"hasawards"] = 0

In [ ]:
df.to_csv("WR-Train.csv",index=False)